In [18]:

from __future__ import print_function
import h5py
from pathlib import Path
from tqdm import tqdm
import numpy as np
from scipy import spatial

import time, pickle
import pandas as pd

from hloc.utils.read_write_model import read_model
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2

from collections import defaultdict
from hloc.patch_utils import patch_co_vis_grouping_new, group_retrival, patch_desc_grouping
from hloc.patch_utils import points_grouping, build_tree, generate_retrieval_file, localize
import metric_learn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sfm_dir = Path("outputs/aachen/sfm_superpoint+NN-superpoint")
global_descriptor_path = Path("common/aachen_v1_1/ref/global-feats-netvlad.h5")
global_descriptor_ref_path = Path("common/aachen_v1_1/test/global-feats-netvlad.h5")
test_image2patch_path = "common/aachen_v1_1/test/image2patch.pkl"
db_image_pairs_path = "pairs/aachen_v1.1/pairs-db-covis20.txt"
image2patch_path = "common/aachen_v1_1/ref/image2patch.pkl"

In [5]:
print(sfm_dir)
cameras, images, points3D = read_model(sfm_dir, ".bin")

outputs/aachen/sfm_superpoint+NN-superpoint


In [6]:
from hloc.utils.parsers import parse_image_list
query_images = parse_image_list(Path("datasets/aachen_v1_1/queries") / 'day_time_queries_with_intrinsics.txt', True)
query_images_night = parse_image_list(Path("datasets/aachen_v1_1/queries") / 'night_time_queries_with_intrinsics.txt', True)
query_images.extend(query_images_night)
query_images[-1]

[04/06/2022 00:12:50 INFO] Imported 824 images from day_time_queries_with_intrinsics.txt
[04/06/2022 00:12:50 INFO] Imported 191 images from night_time_queries_with_intrinsics.txt


('query/night/nexus5x_additional_night/IMG_20170702_005927.jpg',
 ('SIMPLE_RADIAL',
  1600,
  1200,
  array([ 1.19991e+03,  8.00000e+02,  6.00000e+02, -3.24314e-02])))

In [7]:
len(query_images)

1015

In [23]:
test_desc_file = h5py.File(global_descriptor_ref_path, 'r')
x2 = list()
test_patch_id2index = dict()
cnt = 0
for ele in tqdm(test_desc_file.keys()):
    x2.append(np.array(test_desc_file[ele]['global_descriptor']))
    test_patch_id2index[int(ele.split('.')[0])] = cnt
    cnt += 1
x2 = np.array(x2)
test_desc_file.close()

100%|██████████| 1223/1223 [00:00<00:00, 2082.85it/s]


In [26]:
import logging
def patch_grouping_test(n_cluster = 3000, method = "desc"):
    tag = method + "_" + str(n_cluster) 
    group_result_dir = Path("group/aachen_patch/"+tag)
    group_result_dir.mkdir(parents=True, exist_ok=True)
    
    with open("./aachen_test_patch_list.pkl", "rb") as f:
        test_patch_list = pickle.load(f)
    with open("./aachen_patch_list.pkl", "rb") as f:
        patch_list = pickle.load(f) 
    
    # stage 1: clustering
    result_path = group_result_dir / "cluster_result.pkl"
    if n_cluster == -1:
        labels = []
        index2patch_id = dict()
        cnt = 0
        for patch in patch_list:
            labels.append(cnt)
            index2patch_id[cnt] = cnt
            cnt += 1
    else:
        if method == "desc":
            labels, index2patch_id = patch_desc_grouping(global_descriptor_path, 
                                                         patch_list, 
                                                         group_result_dir, 
                                                         n_cluster)
        else:
            labels, index2patch_id = patch_co_vis_grouping_new(patch_list, 
                                                               image2patch_path, 
                                                               db_image_pairs_path,
                                                               group_result_dir,
                                                               n_cluster)
    
    # stage 2: dimensional reduction
    groups_image_points, group_descriptor = points_grouping(patch_list, images, labels, 
                                                            index2patch_id, global_descriptor_path)
    
    tree, index2name = build_tree(group_descriptor)


    for topK in [2, 3, 5, 10]:
#         test_retrieve_groups_file = group_result_dir / str("retrieval_" + str(n_cluster) +"_"+ str(topK)+".txt")
        
#         file = open(test_retrieve_groups_file, "w")

        retrieve_groups = dict()
    
        test_desc_file = h5py.File(global_descriptor_ref_path, 'r')
        for ele in tqdm(test_desc_file.keys()):
            patch_id = int(ele.split('.')[0])
            a = pred[test_patch_id2index[patch_id]]
            ind = np.argpartition(a, -topK)[-topK:]
            ind = ind[np.argsort(a[ind])]
            retrieve_groups[patch_id] = ind
 
        full_tag = tag + "_" + str(topK)
#         generate_retrieval_file(global_descriptor_path, tree, index2name, 
#                                 query_images, group_result_dir, label = str(n_cluster), top_n = topK)
#         retrieve_groups = group_retrival(global_descriptor_ref_path, 
#                                          tree, index2name, k = topK, gt = None)
        
        point_desc_path = Path("mid_data/aachen_v1_1/superpoint_aachen_NN/fused_feats-superpoint-n4096-r1024.pkl")
        local_feature_path = Path("mid_data/aachen_v1_1/superpoint_aachen_superglue/feats-superpoint-n4096-r1024.h5")
        
        result, max_inlier_nums = localize(query_images, 
                                           retrieve_groups, 
                                           local_feature_path, 
                                           point_desc_path,
                                           test_image2patch_path,
                                           groups_image_points, 
                                           points3D, images, cameras,
                                          log_path = str(group_result_dir)+ '/' + full_tag + '.json')
        
        with open(str(group_result_dir)+ "/" + full_tag + ".txt", "w+") as f:
            for name, res in tqdm(result.items()):
                name = name.split('/')[-1]
                f.write(' '.join([name, str(res[0][0]), str(res[0][1]),  str(res[0][2]), str(res[0][3]), 
                                  str(res[1][0]),str(res[1][1]),str(res[1][2])]) + '\n')

In [28]:
# patch_grouping_test(n_cluster = -1, method = "desc")
patch_grouping_test(n_cluster = 1000, method = "co-vis")
patch_grouping_test(n_cluster = 3000, method = "co-vis")
patch_grouping_test(n_cluster = 2000, method = "co-vis")
patch_grouping_test(n_cluster = 1000, method = "co-vis")
patch_grouping_test(n_cluster = 500, method = "co-vis")
patch_grouping_test(n_cluster = 3000, method = "desc")
patch_grouping_test(n_cluster = -1, method = "desc")
patch_grouping_test(n_cluster = 2000, method = "desc")
patch_grouping_test(n_cluster = 1000, method = "desc")
patch_grouping_test(n_cluster = 500, method = "desc")

100%|██████████| 1000/1000 [00:00<00:00, 3972.02it/s]


                  0
count   1000.000000
mean    3013.902000
std     4039.905545
min        1.000000
25%      569.000000
50%     1322.000000
75%     3930.250000
max    33922.000000


100%|██████████| 1223/1223 [00:00<00:00, 28048.87it/s]


localizing: group/aachen_patch/co-vis_1000/co-vis_1000_2.json


100%|██████████| 1223/1223 [00:00<00:00, 42524.40it/s]


...

100%|██████████| 1223/1223 [00:00<00:00, 39886.12it/s]


localizing: group/aachen_patch/desc_500/desc_500_5.json


100%|██████████| 1223/1223 [00:00<00:00, 53383.64it/s]


localizing: group/aachen_patch/desc_500/desc_500_10.json


100%|██████████| 1015/1015 [00:00<00:00, 87485.48it/s]


## descriptor based 

In [65]:
with open("./aachen_test_patch_list.pkl", "rb") as f:
    test_patch_list = pickle.load(f)
with open("./aachen_patch_list.pkl", "rb") as f:
    patch_list = pickle.load(f) 

In [66]:
# method = "co-vis"
method = "desc"

n_cluster = 1000
tag = method + "_" + str(n_cluster) 
group_result_dir = Path("group/aachen_patch/"+tag)
result_path = group_result_dir / "cluster_result.pkl"
with open(result_path, "rb") as f:
    labels = pickle.load(f) 
result_path = group_result_dir / "index2patch_id.pkl"
with open(result_path, "rb") as f:
    index2patch_id = pickle.load(f)     

groups_image_points, group_descriptor = points_grouping(patch_list, images, labels, 
                                                        index2patch_id, global_descriptor_path)

tree, index2name = build_tree(group_descriptor)

100%|██████████| 1000/1000 [00:00<00:00, 4395.33it/s]


                  0
count   1000.000000
mean    3938.366000
std     2963.721391
min       31.000000
25%     1885.000000
50%     3185.500000
75%     4970.500000
max    24349.000000


100%|██████████| 1000/1000 [00:00<00:00, 888435.50it/s]


In [68]:
counter = defaultdict(list)
for i, label in enumerate(labels):
    counter[label].append(index2patch_id[i])
counter_sort = list(counter.items())
counter_sort.sort(key=lambda x:len(x[1]))
show_labels = [x[0] for x in counter_sort[-10:]]

for label in show_labels:
    print(label, counter[label], len(counter[label]))

176 [2255, 2261, 3755, 3798, 3927, 4716, 5119, 5157, 5605, 5657, 5672, 5773, 5877, 6405, 6412, 6417, 6420, 6421, 6554, 6576, 8153] 21
153 [241, 391, 879, 1436, 1445, 1465, 1466, 1472, 1473, 2205, 2234, 2711, 4102, 4360, 4442, 5138, 5191, 5240, 5834, 7133, 7319, 7543] 22
56 [3186, 3590, 3604, 3631, 3733, 3738, 3936, 4328, 4931, 5101, 5132, 5936, 5960, 6016, 6450, 6507, 6515, 6567, 6593, 6600, 6609, 6629] 22
158 [191, 368, 703, 746, 748, 785, 808, 812, 1005, 1008, 1041, 1058, 1752, 1847, 1978, 1990, 3522, 5239, 6389, 6639, 7350, 7898, 8377] 23
68 [674, 1710, 3083, 3086, 3253, 3295, 3296, 3331, 3345, 3362, 3363, 8074, 8207, 8211, 8224, 8225, 8237, 8243, 8256, 8275, 8276, 8288, 8315, 8330] 24
665 [261, 1206, 1286, 1293, 1296, 2270, 2664, 2669, 2670, 2751, 2757, 2831, 3075, 3081, 3168, 3211, 4409, 6875, 6966, 7365, 7660, 7710, 8040, 8173, 8186, 8227, 8290, 8324] 28
985 [353, 1211, 1973, 1986, 2192, 2274, 2277, 2305, 2416, 2472, 2590, 2855, 2952, 3173, 3272, 3277, 4241, 4385, 4651, 4759, 524

In [33]:
retrieve_groups = group_retrival(global_descriptor_ref_path, 
                                 tree, index2name, k = 3, gt = None)

100%|██████████| 1223/1223 [01:11<00:00, 17.08it/s]


In [45]:
with open(test_image2patch_path, "rb") as f:
    test_image2patch = pickle.load(f)

patch_ids = test_image2patch['IMG_20161227_162953.jpg']

# point set merging
for patch_id in patch_ids:
    groups = retrieve_groups[patch_id]
    print(patch_id, groups)
    for group in groups:
        print(group, counter[group])

161 [438, 912, 498]
438 [3757, 4577, 4586, 4965, 5531, 5601, 5709, 5731, 5735, 5826, 5827, 5940, 5941, 5950, 5999, 6033, 6040, 6226, 6227, 6266, 6269, 6534, 6536, 6544, 6550, 7323]
912 [5623]
498 [2126, 3353, 3355]


In [58]:
test_desc = dict()
#     test_desc_file = h5py.File(global_feature_path, 'r')['query']
test_desc_file = h5py.File(global_descriptor_path, 'r')

for ele in tqdm(test_desc_file.keys()):
    patch_id = int(ele.split('.')[0])
    test_desc[patch_id]=np.array(test_desc_file[ele]['global_descriptor'])


100%|██████████| 8633/8633 [00:03<00:00, 2344.57it/s]


In [51]:
std_res = dict()

In [61]:
# compute pairwise distance
import scipy.spatial.distance

def variance(method, n_cluster):
    tag = method + "_" + str(n_cluster) 
    group_result_dir = Path("group/aachen_patch/"+tag)
    result_path = group_result_dir / "cluster_result.pkl"
    with open(result_path, "rb") as f:
        labels = pickle.load(f) 
    result_path = group_result_dir / "index2patch_id.pkl"
    with open(result_path, "rb") as f:
        index2patch_id = pickle.load(f)   
        
    counter = defaultdict(list)
    for i, label in enumerate(labels):
        counter[label].append(index2patch_id[i])

    sum = 0
    cnt = 0

    for label, ids in tqdm(counter.items()):
        vectors=list()
        if len(ids) == 1:
            continue
        cnt += 1
        for patch_id in ids:
            vectors.append(test_desc[patch_id])
    #         vectors.append(position[image_name])
        vectors = np.array(vectors)
        dist = scipy.spatial.distance.cdist(vectors, np.mean(vectors, axis=0).reshape((1,4096)), metric='cosine')
#         print(dist.shape)
        sum += np.mean(dist)

    print(sum/n_cluster, n_cluster)

    # covis_std = sum/cnt
    return sum/n_cluster

In [64]:
for method in ['co-vis', 'desc']:
    for num in ['500','1000', '2000', '3000']:
        std_res[method + '_' + num] = variance(method, int(num))

for method in ['co-vis', 'desc']:
    for num in ['500','1000', '2000', '3000']:
        print(method, num, std_res[method + '_' + num] )


100%|██████████| 500/500 [00:00<00:00, 2336.03it/s]


0.22499210431727884 500


100%|██████████| 1000/1000 [00:00<00:00, 3883.15it/s]


0.2338877283454188 1000


100%|██████████| 2000/2000 [00:00<00:00, 7142.66it/s]


...

0.35106826081624365 1000


100%|██████████| 2000/2000 [00:00<00:00, 5916.96it/s]


0.2574210204949194 2000


100%|██████████| 3000/3000 [00:00<00:00, 8923.51it/s]

0.1772491984446292 3000
co-vis 500 0.22499210431727884
co-vis 1000 0.2338877283454188
co-vis 2000 0.19591578832803624
co-vis 3000 0.15375002463875234
desc 500 0.4237540465149931
desc 1000 0.35106826081624365
desc 2000 0.2574210204949194
desc 3000 0.1772491984446292


In [8]:
group_result_dir = Path("group/aachen")
group_result_dir.mkdir(parents=True, exist_ok=True)

In [9]:
result_path = group_result_dir / "2000_cluster_result_amm.pkl"
labels, index2image_id = image_desc_grouping(global_descriptor_path, images, result_path, n_cluster = 2000)

new


100%|██████████| 6697/6697 [00:06<00:00, 1077.15it/s]


6697


In [90]:
affinity_matrix = np.load("group/aachen/patch_a_matrix.npy")
print(affinity_matrix[3], affinity_matrix == 0, np.sum(affinity_matrix[0]))

[0.         0.15394243 0.         ... 0.         0.         0.        ] [[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]] 2.021410902812332


In [92]:
group_result_dir = Path("group/aachen_patch/covis_3000")
result_path = group_result_dir / "cluster_result.pkl"
db_desc = h5py.File(global_descriptor_path, 'r')

with open(result_path, "rb") as f:
    labels = pickle.load(f)

counter = defaultdict(int)

for i, label in enumerate(labels):
    counter[label] += 1
    if label == 38:
        print(i)
counter = counter.items()
counter = sorted(counter, key = lambda x:x[1], reverse = True) 

show_labels = []
for label, num in counter[:5]:
    print(label, num)
    show_labels.append(label)
    
visualization(images, labels, show_labels, centers = None)
db_desc.close()

41
71
73
78
98
135
141
159
176
180
216
221
225
229
236
240
242
245
253
286
292
300
319
326
328
334
338
339
346
370
379
383
401
421
460
464
465
468
470
475
494
501
517
519
522
524
537
539
547
551
569
581
585
597
614
622
671
697
701
752
790
816
820
822
848
898
924
936
964
967
976
987
996
999
1492
1517
1527
1529
1536
1545
1551
1560
1570
1571
1573
1590
1598
1620
1621
1623
1646
1651
1668
1671
1679
1709
1720
1740
1753
1762
1766
1844
1881
1946
2045
2049
2127
2128
2144
2145
2150
2151
2159
2160
2163
2167
2229
2306
2517
2518
2523
2524
2532
2533
2551
2552
2619
2698
2703
2719
2872
2979
2992
3030
3078
3085
3118
3130
3156
3282
3346
3555
3601
3623
3635
3641
3682
3696
3705
3710
3711
3784
3786
3788
3803
3816
3872
3878
3882
3913
3915
3925
3930
3933
3947
3950
3978
4013
4014
4039
4065
4081
4084
4153
4167
4182
4183
4238
4267
4268
4281
4284
4287
4304
4334
4341
4354
4355
4356
4394
4395
4418
4432
4475
4490
4492
4529
4538
4565
4597
4613
4647
4680
4689
4709
4719
4753
4754
4784
4793
4801
4804
4823
4895
4899
4904

NameError: name 'visualization' is not defined

In [12]:
db_desc = h5py.File(global_descriptor_path, 'r')
with open(result_path, "rb") as f:
    labels = pickle.load(f)
show_labels = show_labels
if show_labels is not None:
    other_label = None
    for i in range(len(labels)):
        if labels[i] not in show_labels:
            if other_label is None:
                other_label = labels[i]
            labels[i] = other_label

class_num = len(set(labels))

group_desc = []
index2image_id = dict()
ind = 0
for image_id, image in tqdm(images.items()):
    group_desc.append(np.array(db_desc[image.name]['global_descriptor']))
    index2image_id[ind] = image_id
    ind += 1

print(len(group_desc))
group_desc=np.array(group_desc)

# tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=1000)
# tsne_results = tsne.fit_transform(group_desc)

x = tsne_results[:,0]
y = tsne_results[:,1]
plt.figure(figsize=(16,16))
sns.scatterplot(
    x=x, y=y,
    hue=labels,
    palette=sns.color_palette("hls", class_num),
    legend="full",
    alpha=0.3
)

100%|██████████| 6697/6697 [00:02<00:00, 2730.59it/s]


6697


NameError: name 'tsne_results' is not defined

## co-visibility based grouping

In [13]:
# group_point_set_path = group_result_dir / "group_point_set.pkl"
# group_desc_path = group_result_dir / "group_desc.pkl"
result_path = group_result_dir / "2000_cluster_result_amm.pkl"
with open(result_path, "rb") as f:
    labels = pickle.load(f)
groups_image_points, group_descriptor = points_grouping(images, labels, index2image_id, global_descriptor_path)

100%|██████████| 2000/2000 [00:00<00:00, 33886.25it/s]


                  0
count   2000.000000
mean    3329.304000
std     1967.417913
min       65.000000
25%     1933.750000
50%     2964.500000
75%     4319.500000
max    15052.000000


In [14]:
tree, index2name = build_tree(group_descriptor)
generate_retrieval_file(global_descriptor_path, tree, index2name, query_images, group_result_dir, label = "2000", top_n = 5)

100%|██████████| 1015/1015 [00:57<00:00, 17.53it/s]


In [39]:
test_retrieve_groups_file = group_result_dir / "2000_5.txt"
point_desc_path = Path("mid_data/aachen_v1_1/superpoint_aachen_superglue/fused_feats-superpoint-n4096-r1024.pkl")
local_feature_path = Path("mid_data/aachen_v1_1/superpoint_aachen_superglue/feats-superpoint-n4096-r1024.h5")

logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

result, max_inlier_nums = localize(query_images, test_retrieve_groups_file, local_feature_path, point_desc_path,
             groups_image_points, points3D, images, cameras)

[03/26/2022 22:09:50 INFO] using retrieval file :group/aachen/2000_5.txt
[03/26/2022 22:09:50 INFO] image based
[03/26/2022 22:09:50 INFO] loading point descs1
[03/26/2022 22:09:55 INFO] loading point descs2
[03/26/2022 22:09:59 INFO] merging
[03/26/2022 22:09:59 INFO] loading retrieval result
[03/26/2022 22:09:59 INFO] load local feature
[03/26/2022 22:09:59 INFO] create matcher


  0%|          | 0/1015 [00:00<?, ?it/s]

[03/26/2022 22:09:59 INFO] ##### localizing: query/day/nexus4/IMG_20140520_182846.jpg
[03/26/2022 22:09:59 INFO] # candidate_point_set: 1
[03/26/2022 22:09:59 INFO] new cps with size  31794
[03/26/2022 22:10:00 INFO] #inliers: 902
[03/26/2022 22:10:00 INFO] #result: True
[03/26/2022 22:10:00 INFO] #num_inliers: 888
[03/26/2022 22:10:00 INFO] ### final: max_inlier_nums: 888


  0%|          | 1/1015 [00:01<17:41,  1.05s/it]

[03/26/2022 22:10:00 INFO] ##### localizing: query/day/nexus4/IMG_20140520_211432.jpg
[03/26/2022 22:10:00 INFO] # candidate_point_set: 1
[03/26/2022 22:10:00 INFO] new cps with size  29967
[03/26/2022 22:10:01 INFO] #inliers: 195
[03/26/2022 22:10:01 INFO] #result: True
[03/26/2022 22:10:01 INFO] #num_inliers: 176
[03/26/2022 22:10:01 INFO] ### final: max_inlier_nums: 176


...

[03/26/2022 22:23:47 INFO] ##### localizing: query/night/nexus5x_additional_night/IMG_20170702_005747.jpg
[03/26/2022 22:23:47 INFO] # candidate_point_set: 5
[03/26/2022 22:23:47 INFO] new cps with size  8611
[03/26/2022 22:23:48 INFO] #inliers: 4
[03/26/2022 22:23:48 INFO] #result: True
[03/26/2022 22:23:48 INFO] #num_inliers: 3
[03/26/2022 22:23:48 INFO] new cps with size  7156
[03/26/2022 22:23:48 INFO] #inliers: 0
[03/26/2022 22:23:48 INFO] #result: False
[03/26/2022 22:23:48 INFO] new cps with size  5437
[03/26/2022 22:23:48 INFO] #inliers: 1
[03/26/2022 22:23:48 INFO] #result: False
[03/26/2022 22:23:48 INFO] new cps with size  2984
[03/26/2022 22:23:48 INFO] #inliers: 7
[03/26/2022 22:23:48 INFO] #result: True
[03/26/2022 22:23:48 INFO] #num_inliers: 5
[03/26/2022 22:23:48 INFO] new cps with size  2747
[03/26/2022 22:23:48 INFO] #inliers: 2
[03/26/2022 22:23:48 INFO] #result: False
[03/26/2022 22:23:48 INFO] ### final: max_inlier_nums: 5


100%|█████████▉| 1014/1015 [13:49<00:00,  1.34it/s]

[03/26/2022 22:23:48 INFO] ##### localizing: query/night/nexus5x_additional_night/IMG_20170702_005927.jpg
[03/26/2022 22:23:48 INFO] # candidate_point_set: 3
[03/26/2022 22:23:48 INFO] new cps with size  9558
[03/26/2022 22:23:49 INFO] #inliers: 13
[03/26/2022 22:23:49 INFO] #result: True
[03/26/2022 22:23:49 INFO] #num_inliers: 7
[03/26/2022 22:23:49 INFO] new cps with size  8611
[03/26/2022 22:23:49 INFO] #inliers: 0
[03/26/2022 22:23:49 INFO] #result: False
[03/26/2022 22:23:49 INFO] new cps with size  3598
[03/26/2022 22:23:49 INFO] #inliers: 0
[03/26/2022 22:23:49 INFO] #result: False
[03/26/2022 22:23:49 INFO] ### final: max_inlier_nums: 7


100%|██████████| 1015/1015 [13:49<00:00,  1.22it/s]

4.605665922164917 561.845276594162 0 65.11111354827881 64.35600447654724 126.81413412094116 829.9219472408295


In [42]:
with open("aachen_test.txt", "w+") as f:
    for name, res in tqdm(result.items()):
        name = name.split('/')[-1]
        f.write(' '.join([name, str(res[0][0]), str(res[0][1]),  str(res[0][2]), str(res[0][3]), 
                          str(res[1][0]),str(res[1][1]),str(res[1][2])]) + '\n')

100%|██████████| 1015/1015 [00:00<00:00, 86751.00it/s]


In [7]:
# fused = h5py.File("mid_data/aachen_v1_1/superpoint_aachen_superglue/fused_feats-superpoint-n4096-r1024_test.h5", 'w')
old = h5py.File("mid_data/aachen_v1_1/superpoint_aachen_superglue/feats-superpoint-n4096-r1024.h5", 'r')

save_path = Path("mid_data/aachen_v1_1/superpoint_aachen_superglue/fused_feats-superpoint-n4096-r1024_2.pkl")
# save_dict = dict()
# cnt = 0
# for pid, point in tqdm(points3D.items()):
#     ave_desc = old[images[point.image_ids[0]].name]['descriptors'][:,point.point2D_idxs[0]]
#     save_dict[pid] = ave_desc
#     cnt += 1
#     if cnt >= 1000000:
#         break
    
# with open(save_path, "wb") as file:
#     pickle.dump(save_dict, file)
    
save_dict = dict() 
cnt = 0
for pid, point in tqdm(points3D.items()):
    cnt += 1
    if cnt < 1000000:
        continue
    ave_desc = old[images[point.image_ids[0]].name]['descriptors'][:,point.point2D_idxs[0]]
    save_dict[pid] = ave_desc  
    

with open(save_path, "wb") as file:
    pickle.dump(save_dict, file)
    
old.close()

100%|██████████| 1804975/1804975 [18:21<00:00, 1638.49it/s]  
